### Set up

Importing the data is taken care by kaggle.<br>
Just upload your kaggle.json API token file(You can get it from your accounts page on Kaggle)

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sharifahrodziahhanim","key":"f748201dff56e6616bbd35b9314955fa"}'}

In [2]:
!pip3 install geopandas
!pip install cartopy
!pip3 install mplleaflet
! pip install -q kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 94.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-py3-none-any.whl size=28565 sha256=33dd71db29c3236fe0b094b38d842c9a379bb9862ffd29476bdcc34f5c2fb639
  Stored in directory: /root/.cache/pip/wheels/19/d8/76/c40290202e76c98889c67d6c21fe5e7057334b088e21ef88ba
Successfully built mplleaflet


In [3]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import mplleaflet as mpll

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                                    title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                      Chocolate Sales Data 📊🍫                                 14473  2025-03-19 03:51:40.270000           8898        146  1.0              
abdulmalik1518/mobiles-dataset-2025                                    Mobiles Dataset (2025)                                  20314  2025-02-18 06:50:24.370000          15240        268  1.0              
mahmoudelhemaly/students-grading-dataset                               Student Performance & Behavior Dataset                 520428  2025-02-17 17:38:46.653000          10933 

In [8]:
!kaggle datasets download -d tsiaras/uk-road-safety-accidents-and-vehicles

Dataset URL: https://www.kaggle.com/datasets/tsiaras/uk-road-safety-accidents-and-vehicles
License(s): DbCL-1.0


In [9]:
! mkdir train
! unzip uk-road-safety-accidents-and-vehicles.zip -d train

Archive:  uk-road-safety-accidents-and-vehicles.zip
  inflating: train/Accident_Information.csv  
  inflating: train/Vehicle_Information.csv  


In [10]:
for dirname, _, filenames in os.walk('train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


train/Vehicle_Information.csv
train/Accident_Information.csv


In [11]:
accidents=pd.read_csv("train/Accident_Information.csv")

<ipython-input-11-0ace3edffbf7>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents=pd.read_csv("train/Accident_Information.csv")


# GNN model

In [13]:
!pip install torch-geometric --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import kneighbors_graph
import numpy as np

# Load CSV Dataset
# acc = pd.read_csv("Accident_Information.csv")

# Extract Features and Labels
features = ['Latitude', 'Longitude', 'Speed_limit', 'Weather_Conditions']  # Add other relevant features
target = 'Accident_Severity'
X = accidents[features].dropna().reset_index(drop=True)
y = accidents[target].dropna().reset_index(drop=True)

# Drop rows with missing values in either features or target before separating X and y
accidents = accidents.dropna(subset=features + [target]).reset_index(drop=True)

X = accidents[features]
y = accidents[target]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for dense output

# Fit and transform the 'Weather_Conditions' column
weather_encoded = encoder.fit_transform(X[['Weather_Conditions']])

# Create column names for the encoded features
weather_cols = [f'Weather_{cat}' for cat in encoder.categories_[0]]

# Create a DataFrame with the encoded features
weather_df = pd.DataFrame(weather_encoded, columns=weather_cols, index=X.index)

# Drop the original 'Weather_Conditions' column and concatenate the encoded features
X = X.drop(columns=['Weather_Conditions']).join(weather_df)

# Encode Labels
y = LabelEncoder().fit_transform(y)

# Convert Data to PyTorch Tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Create Edges using Spatial Nearest Neighbors (KNN Graph)
n_neighbors = 5  # Adjust based on road network density
adj_matrix = kneighbors_graph(X[['Latitude', 'Longitude']], n_neighbors, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)

# Define the GNN Model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = pyg_nn.GCNConv(input_dim, hidden_dim)
        self.conv2 = pyg_nn.GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize Model, Loss, and Optimizer
input_dim = X.shape[1]
hidden_dim = 16
output_dim = len(set(y))

model = GNNModel(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train the Model
def train(model, X, y, edge_index, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(X, edge_index)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

train(model, X_tensor, y_tensor, edge_index, epochs=10)

# Evaluate the Model
def evaluate(model, X, y, edge_index):
    model.eval()
    with torch.no_grad():
        out = model(X, edge_index)
        predictions = out.argmax(dim=1)
        accuracy = (predictions == y).sum().item() / y.size(0)
        print(f"Test Accuracy: {accuracy:.4f}")

evaluate(model, X_tensor, y_tensor, edge_index)


Epoch 1/10, Loss: 5.6910
Epoch 2/10, Loss: 4.7976
Epoch 3/10, Loss: 3.9301
Epoch 4/10, Loss: 3.0824
Epoch 5/10, Loss: 2.2469
Epoch 6/10, Loss: 1.4530
Epoch 7/10, Loss: 1.5506
Epoch 8/10, Loss: 1.9549
Epoch 9/10, Loss: 1.6413
Epoch 10/10, Loss: 1.1100
Test Accuracy: 0.8399
